In [1]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
import inflect #numeric/cardinal

In [101]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [102]:
dataFile = gzip.open(path, 'r')

In [103]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [104]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [105]:
len(productReviews)

134476

In [106]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [107]:
productsReviewData.shape
productsReviewData.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [108]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]




In [109]:
rating = lambda x: 1 if x > 2.0 else 0

In [110]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews

In [111]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [112]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [113]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(134476, 3)

In [15]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [114]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



dict_keys(["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't", "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's", "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's", "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've", "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's", 's###', "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's", "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't", "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've", "when's", "when've", "where

In [23]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [115]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [116]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [117]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 3min 16s


In [20]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [27]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [28]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [29]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [30]:
#print(stopWords)

In [31]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [118]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [119]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 6min 36s


In [120]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 201 ms


In [826]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']]
#reviewTextProcessing.dtypes

,Processed,Sentiment,SentimentModel
5494,"[it sounds very artificial i remember first getting this pedal and really loving the sound and kick it had now that i use it with a better amp it sounds very processed and artificial in sound that it has it is not even worth the price i payed for it before, i also had bought another one to replace the first one because the input section jack got messed up in the amp that it did not want to lock with my guitar cable and when i took it apart there was no way for me to fix it other than sending it to be replaced, shitty pedal]","[0.1411111111111111, 0.0625, 0.0]","[0.2036111111111111, 1.0]"
3792,"[i bought this because hey a wireless mic for, why not, i might be able to use this in a public address system the connections all work and it makes sounds but the static is unbearable no matter how much i fiddle with different the adjustment screw in the receiver nothing works, the audio is always clipped, don t even think of using this for music]","[0.0, 0.0, 0.24, 0.0, 0.0]","[0.24, 1.0]"
6090,"[in an attempt to see what has what is new in the pick industry i decided to dive in and give these a go, they are way too flimsy if like me you use a heavier pick, i have always used dunlop tortex greens and these davas just can not compete, not only were they too flimsy but when i played with any aggression heavy rock based material the picks would fly out of my hands, this was a huge disappointment]","[0.13636363636363635, 0.0, 0.0, 0.20000000000000004, -0.09999999999999995]","[0.23636363636363647, 1.0]"
3748,"[hard to believe this was so bad, i have a planet waves locking guitar strap which is a dream and thought this would allow me to lock a regular strap, however it is so thick that it can not really fit on a regular guitar strap pin with a strap and when i tried to lock it the two pieces came apart, go with the planet waves lock straps]","[-0.49583333333333324, 0.0, -0.16666666666666666, 0.0]","[-0.6624999999999999, 0.0]"
6077,"[sound nice but very unreliable bx a deluxe i own a pair of yamaha ns s and event s and they are all currently working without problems with over years use years ago i purchased a set of regular bx a s and they lasted about years and then the faulty capacitors leaked i just replaced them with the bx a deluxe model and after only one week, one of the internal amps will no longer power up i need a pair of affordable av shielded monitors that i can depend on and these cheap m audio monitors are not it i am back to using my s with slight av noise until i find a decent av shielded monitor m audio was a much better company before avid took them over]","[0.3333333333333333, 0.16428571428571428]","[0.4976190476190476, 1.0]"
3746,"[i even called dunlop to see if these lok strap systems would work for an electric acoustic, the guy at dunlop told me oh yes they have been specifically designed to work with electric acoustic, that is not the case, i purchased multiple units and it will cost me more to return to amazon than it has it is worth, but basically i know have hunks of plastic that i could use to make a sculpture because of the bottom pin being the plug in for electric pick up the thickness causes a problem and this plastic do hickey will not fit if anyone out there has a solution for securing the strap please reply to this feedback what really frosts me is that i called dunlop and apparently their customer service folks do not know their products well enough to be recommending product, it s called train your people, ]","[0.0, 0.0, 0.0, 0.26666666666666666, 0.0125, 0.0, 0.0]","[0.2791666666666667, 1.0]"
1870,"[i have used three pop filters in my experiences with home recording and podcasting, this one is the worst, the plastic screw yes plastic that is meant to grip the pole that you attach the pop filter to is completely incapable of supporting it has it is own weight at nearly any angle, bad just bad secondly there is a metal piece that secures the flex arm to the clamp, it s

In [121]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [122]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'Sentiment',
       'SentimentModel', 'CleanedJoin'],
      dtype='object')

In [123]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [124]:
reviewTextProcessing['CleanedJoin'].head(5)

0                            i hate it when my shirt collars not otherwise secured in place by buttons end up in weird places throughout the day, i purchased some steel collar stays to use with these magnets but they were only vaguely magnetic, i ended up using of these magnets one in the collar with the stay and the other inside my shirt to lock my collar in place, they work flawlessly, they are the perfect size and there are plenty of magnets in case you forget to remove them at the end of the day
1                                                                                                                                                                                                                                                                                                                                                              these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about o

In [125]:

import itertools

largeText = reviewTextProcessing.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

134476

In [126]:
sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [127]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 10000


In [128]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [129]:

size = int(len(largeText)*.75)
x_train_text = largeText[:size]
x_test_text = largeText[size:]

y_train =  reviewTextProcessing['rating'][:size]
y_test = reviewTextProcessing['rating'][size:]
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [130]:
len(y_train), len(x_train_tokens), len(x_train_text), len(x_test_text)

(100857, 100857, 100857, 33619)

In [131]:
#x_train_tokens[1], x_train_text[1]

In [132]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [133]:
np.mean(num_tokens), np.max(num_tokens)


(110.80553407299443, 4838)

In [134]:
np.std(num_tokens)

132.92503796578936

In [135]:
max_tokens = np.mean(num_tokens) + 1 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


243

In [136]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.897914869567804

In [137]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape
x_test_pad.shape


(33619, 243)

In [138]:
for i in range(0, 1):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    2 1558
    4   39   18 3329 8531   11  722 2917   12  236   82 1219  220   41
   12 2405  908 2048    1  225    2  212   68  424 2317  951    5   32
   17   37 1753   19   27  120   64 1190    2  996   41   96    8   37
 1753 

In [139]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [140]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [141]:
x_train_text[1]


'these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of a pound'

In [142]:
tokens_to_string(x_train_tokens[1])


'these little magnets are really powerful for there size i am using them to make secret compartments in custom made boxes each one about of a pound'

In [143]:
from tensorflow.python.keras import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam

In [144]:
import tensorflow as tf
tf.__version__


'1.13.2'

In [145]:
optimizer = Adam(lr=1e-3)


In [146]:
model_cnn_02 = Sequential()
e = Embedding(num_words, 200, input_length=max_tokens)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_cnn_02.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 243, 200)          2000000   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 242, 100)          40100     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 2,066,213
Trainable params: 2,066,213
Non-trainable params: 0
_________________________________________________________________


In [147]:
%%time
model_cnn_02.fit(x_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=32)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95814 samples, validate on 5043 samples
Epoch 1/10
95814/95814 [==============================] - 372s 4ms/step - loss: 0.1841 - acc: 0.9344 - val_loss: 0.1623 - val_acc: 0.9427
Epoch 2/10
95814/95814 [==============================] - 372s 4ms/step - loss: 0.1310 - acc: 0.9507 - val_loss: 0.1582 - val_acc: 0.9415
Epoch 3/10
95814/95814 [==============================] - 367s 4ms/step - loss: 0.0961 - acc: 0.9631 - val_loss: 0.1736 - val_acc: 0.9425
Epoch 4/10
95814/95814 [==============================] - 364s 4ms/step - loss: 0.0638 - acc: 0.9760 - val_loss: 0.2162 - val_acc: 0.9405
Epoch 5/10
95814/95814 [==============================] - 366s 4ms/step - loss: 0.0382 - acc: 0.9863 - val_loss: 0.2674 - val_acc: 0.9413
Epoch 6/10
95814/95814 [==============================] - 365s 4ms/step - loss: 0.0248 - acc: 0.9914 - val_loss: 0.2987 - val_acc: 0.9371
Epoch 7/10
95814/95814 [==============================] - 366s 4ms/step - loss: 0.0169 - acc: 0.9944 - val_loss: 0.3489 - v

In [148]:
%%time
result = model_cnn_02.evaluate(x_test_pad, y_test)

33619/33619 [==============================] - 29s 856us/step
Wall time: 28.8 s


In [149]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 92.57%


In [150]:
%%time
y_pred = model_cnn_02.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

Wall time: 819 ms


In [151]:
cls_pred = np.array([1.0 if p>0.2 else 0.0 for p in y_pred])


In [152]:
cls_true = np.array(y_test[0:1000])


In [162]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
incorrect

array([ 37,  39,  45,  85, 107, 148, 190, 197, 204, 281, 297, 365, 418,
       445, 450, 454, 521, 522, 537, 542, 546, 570, 588, 600, 617, 631,
       633, 658, 661, 663, 675, 676, 679, 712, 723, 735, 737, 757, 763,
       795, 816, 864, 886, 905, 914, 922, 940, 942, 944, 949, 957, 965,
       970, 992], dtype=int64)

In [160]:
correct = np.where(cls_pred == cls_true)
correct[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  38,  40,
        41,  42,  43,  44,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  86,  87,  88,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

In [154]:
len(incorrect)


54

In [98]:
reduceData.loc[7695+40, ['reviewText', 'overall', 'rating']]
reviewTextProcessing.loc[7695+40, ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

CleanedJoin       so heres the thing that no one else wants to say, the thing is built like a piece of shit, it jingles and rattles and parts are always moving around, i used this times and it shorted out, the lights started flashing and it got hot, i called line and they determined it was bad so they opened a ticket, fyi if you want to deal with the worst customer service in the industry just call line not only do they not know what they were talking about but in the backgrounc i could hear them lauging and ...
overall                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [155]:
idx = incorrect[1]
print(type(idx))
for each in incorrect[:10]:
    idx = each
    text = x_test_text[idx]
    print(idx, text, cls_pred[idx], cls_true[idx], " BlobText ",
         TextBlob(text).sentiment.polarity)


<class 'numpy.int64'>
37 i purchased this alarm months ago after two waters flooded my home within year, after the current water heater was installed i installed this alarm with the sensor on the floor around the heater, the alarm went off once, i had a very tiny leak, i was very impressed that this alarm worked with just a tiny drop of water on the floor, great, it turned out that i needed an overflow tank so my plumber installed that and put the alarm back in place, that was the end of nov or early dec this morning at am the alarm sounded, using paper towels i checked the floor and wiped with paper towels no moisture wiped off the cable to the alarm and put in place, when i plugged the cord into the alarm the alarm sounded and it will not stop, i called honeywell tech support, after having me wipe the floor and cable again paper towel was completely dry i was asked to remove the battery and wait minutes, the tech person told me to plug in the alarm again, the alarm sounded, i was hop

In [173]:
reviewTextProcessing.loc[list(incorrect + 95814 + 5043), ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]
reviewTextProcessing.loc[[101061, 101054, 101047, 101163
                         ], ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

,CleanedJoin,overall,rating,SentimentModel
101061,"i love amazon, i was thinking how nice the rockwell would be to own, after reading the reviews, i will wait until they improve it since it appears to be junk was it made in china",2.0,0,"[1.1, 1.0]"
101054,"definitely not good for ripping even one inch stock in spite of what the description says, it was impossible to keep the blade from being distorted although i used the fence and was slow and careful, the blade is just going to be prone to doing that because it has it is basically a reciprocating saw blade rather long and flexible vs a more dimensionally stable circular blade such as a table saw has, this saw could be useful for straight cuts in thinner stock but my miter saw and band saw han...",3.0,1,"[-0.9055555555555554, 0.0]"
101047,"for the most part this machine is a good compromise that does a fairly good job but not great one, it s great for hobby small wood working projects but nothing on a larger scale, one problem i have is with the miter gauge, it does not slide in the channels smoothly and does not hold a setting very well, it s actually very cheaply made and not adequate for a job that requires any degree of accuracy, in searching for a better after market miter gauge i found that rockwell has designed this saw...",1.0,0,"[1.8850000000000002, 1.0]"
101163,"i did not buy this unit but i did buy and return the similar honeywell pls c these claim to be three way switches but require four wires, just a note if you plan to replace this yourself check your wiring first",1.0,0,"[0.25, 1.0]"


In [172]:
134476 - (33619 + 95814)
95814 + 5043 + 306

101163

In [82]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)